In [3]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score
from plotly.subplots import make_subplots
import statsmodels.api as sm

Using TensorFlow backend.


In [4]:
data = pd.read_csv("dailySummaries.csv")
data = data.set_index("day")
data = data[data.columns[1:]]
data = data.loc[:"2022-01-06 00:00:00"]

# train_weight = 0.80 # Using 80% of data for training, 20% for testing
# split = int(len(data)*train_weight)
# df_train = data.iloc[:split].copy()
# df_test = data.iloc[split:].copy()

In [3]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]

y_train = df_train["aaveHourlyPctChange"]
y_test = df_test["aaveHourlyPctChange"]

In [4]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [5]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [6]:
adf, p, usedlag, nobs, cvs,aic=sm.tsa.stattools.adfuller(y_train)

In [7]:
# Null hypothesis: Non Stationarity exists in the series.
# Alternative Hypothesis: Stationarity exists in the series

print("Critical Value: ", adf)
print("p-value" , p)
print("T-values for adfuller: ", cvs)
# Reject the null -- stationarity exists

Critical Value:  -14.847660026101146
p-value 1.797020021847359e-27
T-values for adfuller:  {'1%': -3.431201534294984, '5%': -2.8619163150009173, '10%': -2.566970307948449}


In [8]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_train.index, y=df_test.aaveHourlyPctChange*100,
                    mode='lines',
                    name='Aave Change'),
                    secondary_y=False)
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)',
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})
fig.update_xaxes(title="Time (by hour)")
fig.update_yaxes(title="Percent Change (%)")

# Alpha RNN

In [9]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [10]:
max_epoches=1000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

In [11]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)

2022-03-24 23:40:33.667567: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-24 23:40:33.669176: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/1000
7719/7719 [==============================] - 6s 814us/step - loss: 0.1753
Epoch 2/1000
7719/7719 [==============================] - 1s 75us/step - loss: 0.0939
Epoch 3/1000
7719/7719 [==============================] - 1s 74us/step - loss: 0.0397
Epoch 4/1000
7719/7719 [==============================] - 1s 75us/step - loss: 0.0152
Epoch 5/1000
7719/7719 [==============================] - 1s 74us/step - loss: 0.0100
Epoch 6/1000
7719/7719 [==============================] - 1s 74us/step - loss: 0.0107
Epoch 7/1000
7719/7719 [==============================] - 1s 75us/step - loss: 0.0103
Epoch 8/1000
7719/7719 [==============================] - 1s 74us/step - loss: 0.0093
Epoch 9/1000
7719/7719 [==============================] - 1s 74us/step - loss: 0.0086
Epoch 10/1000
7719/7719 [==============================] - 1s 76us/step - loss: 0.0083
Epoch 11/1000
7719/7719 [==============================] - 1s 76us/step - loss: 0.0080
Epoch 12/1000
7719/7719 [==========================

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_1 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [16]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred, multioutput='variance_weighted')

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

adj_r2 = 1 - (1-r2_test)*(len(y_test_reg)-1)/(len(y_test_reg)-x_test_reg.shape[1]-1)
print("Adjusted r2: ", adj_r2)

7719/7719 [==============================] - 1s 156us/step
Train MSE:  0.00024608938836608886
Train r2:  -0.00971288697645023
1930/1930 [==============================] - 0s 129us/step
Test MSE:  0.00010930533979914673
Test r2:  -0.02921103512208445
Adjusted r2:  -0.04602112052186569


In [14]:
pred_prices = []
for i in range(1,len(df_test)):
    price = df_test.aavePriceUSD.iloc[i-1] * (1 + y_pred[i])
    pred_prices.append(price[0])

In [15]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_test.index, y=pred_prices,
                    mode='lines',
                    marker=dict(color='red'),
                    name='Aave Predicted'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.aavePriceUSD,
                    mode='lines',
                    marker=dict(color='blue'),
                    name='Aave Actual'),
                    secondary_y=False)
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)',
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})
fig.update_xaxes(title="Time (by hour)")
fig.update_yaxes(title="Price (USD)")